In [13]:
import transformer_lens.utils as utils
from transformer_lens import HookedTransformer, HookedTransformerConfig
# from mech_interp_othello_utils import OthelloBoardState
import einops
import torch
from tqdm import tqdm
import numpy as np
from fancy_einsum import einsum
import chess
import numpy as np
import csv
import chess_utils
from dataclasses import dataclass

device = "cuda"
device = "cpu"
device = "mps"

n_layers = 16
n_heads = 8
MODEL_DIR = "models/"
DATA_DIR = "data/"
cfg = HookedTransformerConfig(
    n_layers = n_layers,
    d_model = 512,
    d_head = 64,
    n_heads = n_heads,
    d_mlp = 2048,
    d_vocab = 32,
    n_ctx = 1023,
    act_fn="gelu",
    normalization_type="LNPre"
)
model = HookedTransformer(cfg)
model_name = f"tf_lens_{n_layers}"
model.load_state_dict(torch.load(f'{MODEL_DIR}{model_name}.pth'))
model.to(device)

Moving model to device:  mps


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-31): 32 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (h

In [14]:
layer = 12

@dataclass
class Config:
    min_val: int
    max_val: int
    custom_function: callable
    linear_probe_name: str

piece_config = Config(
    min_val = -6,
    max_val = 6,
    custom_function = chess_utils.board_to_piece_state,
    linear_probe_name = "chess_piece_probe",
)

color_config = Config(
    min_val = -1,
    max_val = 1,
    custom_function=chess_utils.board_to_piece_color_state,
    linear_probe_name="chess_color_probe",
)

random_config = Config(
    min_val = -1,
    max_val = 1,
    custom_function=chess_utils.board_to_random_state,
    linear_probe_name="chess_random_probe",
)

config = piece_config
# config = color_config
# config = random_config

In [15]:
board_seqs_int = torch.tensor(np.load(f"{DATA_DIR}train_board_seqs_int.npy")).long()
print(board_seqs_int.shape)
dots_indices = torch.tensor(np.load(f"{DATA_DIR}train_dots_indices.npy")).long()
# state_stack = torch.tensor(np.load("state_stacks_5k.npy")).long()
print(dots_indices.shape)
# print(state_stack.shape)

board_seqs_string = []

with open(f"{DATA_DIR}train_board_seqs_string.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        board_seqs_string.append(row[0])
print(len(board_seqs_string), len(board_seqs_string[0]))
# print(board_seqs_string[0])


torch.Size([32965, 680])
torch.Size([32965, 61])
32965 680


In [16]:
# print(board_seqs_string[0])
# custom_function = chess_utils.board_to_piece_state
custom_function = config.custom_function
print(chess_utils.create_state_stack(board_seqs_string[0], custom_function).shape)

state_stack = torch.tensor(chess_utils.create_state_stack(board_seqs_string[0], custom_function)).long()
print(state_stack.shape)

state_stacks = chess_utils.create_state_stacks(board_seqs_string[:50], custom_function)
print(state_stacks.shape)

(680, 8, 8)
torch.Size([680, 8, 8])
torch.Size([1, 50, 680, 8, 8])


In [17]:
batch_size = 1
max_lr = 2e-4
wd = 0.01
pos_start = 5 # indexes into white_moves_indices or dot_indices
# pos_end = model.cfg.n_ctx - 5
# input_length = 680
# pos_end = input_length - 0
# length = pos_end - pos_start
one_hot_range = config.max_val - config.min_val + 1
rows = 8
cols = 8
num_epochs = 1
num_games = 10000
x = 0
y = 2
# The first mode is blank or not, the second mode is next or prev GIVEN that it is not blank
modes = 1
# alternating = torch.tensor([1 if i%2 == 0 else -1 for i in range(length)], device=device)


state_stack_one_hot = chess_utils.state_stack_to_one_hot(modes, rows, cols, config.min_val, config.max_val, device, state_stacks)
print(state_stack_one_hot.shape)
print((state_stack_one_hot[:, 1, 170, 4:9, 2:5]))
print((state_stacks[:, 1, 170, 4:9, 2:5]))

torch.Size([1, 50, 680, 8, 8, 13])
tensor([[[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
          [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]]], device='mps:0')
tensor([[[ 0, -3, -1],
         [-2, -1,  0],
         [ 0,  0,  0],
         [ 0,  0, -6]]])


In [18]:
max_lr = 3e-4
min_lr = max_lr / 10
max_iters = num_games * num_epochs
decay_lr = True

def get_lr(current_iter: int, max_iters: int, max_lr: float, min_lr: float) -> float:
    """
    Calculate the learning rate using linear decay.

    Args:
    - current_iter (int): The current iteration.
    - max_iters (int): The total number of iterations for decay.
    - lr (float): The initial learning rate.
    - min_lr (float): The minimum learning rate after decay.

    Returns:
    - float: The calculated learning rate.
    """
    # Ensure current_iter does not exceed max_iters
    current_iter = min(current_iter, max_iters)

    # Calculate the linearly decayed learning rate
    decayed_lr = max_lr - (max_lr - min_lr) * (current_iter / max_iters)

    return decayed_lr

def train_linear_probe(layer: int):
    linear_probe_name = f"{MODEL_DIR}{model_name}_{config.linear_probe_name}_layer_{layer}.pth"
    linear_probe = torch.randn(
        modes, model.cfg.d_model, rows, cols, one_hot_range, requires_grad=False, device=device
    )/np.sqrt(model.cfg.d_model)
    linear_probe.requires_grad = True
    print(linear_probe.shape)
    lr = max_lr
    optimiser = torch.optim.AdamW([linear_probe], lr=lr, betas=(0.9, 0.99), weight_decay=wd)

    print(dots_indices.shape)

    # print(dots_indices.shape)


    current_iter = 0
    loss = 0
    for epoch in range(num_epochs):
        full_train_indices = torch.randperm(num_games)
        for i in tqdm(range(0, num_games, batch_size)):

            lr = get_lr(current_iter, max_iters, max_lr, min_lr) if decay_lr else lr
            for param_group in optimiser.param_groups:
                param_group['lr'] = lr
            
            indices = full_train_indices[i:i+batch_size]
            list_of_indices = indices.tolist() # For indexing into the board_seqs_string list of strings
            # print(list_of_indices)
            games_int = board_seqs_int[indices]
            games_int = games_int[:, :]
            # print(games_int.shape)
            games_str = [board_seqs_string[idx] for idx in list_of_indices]
            games_str = [s[:] for s in games_str]
            games_dots = dots_indices[indices]
            games_dots = games_dots[:, pos_start:]
            # print(games_dots.shape)
            state_stack = chess_utils.create_state_stacks(games_str, custom_function)
            # state_stack = state_stack[:, pos_start:pos_end, :, :]
            # print(state_stack.shape)
            # Initialize a list to hold the indexed state stacks
            indexed_state_stacks = []

            for batch_idx in range(state_stack.size(0)):
                # Get the indices for the current batch
                dots_indices_for_batch = games_dots[batch_idx]

                # Index the state_stack for the current batch
                indexed_state_stack = state_stack[:, batch_idx, dots_indices_for_batch, :, :]

                # Append the result to the list
                indexed_state_stacks.append(indexed_state_stack)

            # Stack the indexed state stacks along the first dimension
            # This results in a tensor of shape [2, 61, 8, 8] (assuming all batches have 61 indices)
            state_stack = torch.stack(indexed_state_stacks)
            # print("after indexing state stack shape", state_stack.shape)

            state_stack_one_hot = chess_utils.state_stack_to_one_hot(modes, rows, cols, config.min_val, config.max_val, device, state_stack).to(device)
            with torch.inference_mode():
                _, cache = model.run_with_cache(games_int.to(device)[:, :-1], return_type=None)
                resid_post = cache["resid_post", layer][:, :]
            # Initialize a list to hold the indexed state stacks
            indexed_resid_posts = []

            for batch_idx in range(games_dots.size(0)):
                # Get the indices for the current batch
                dots_indices_for_batch = games_dots[batch_idx]

                # Index the state_stack for the current batch
                indexed_resid_post = resid_post[batch_idx, dots_indices_for_batch]

                # Append the result to the list
                indexed_resid_posts.append(indexed_resid_post)

            # Stack the indexed state stacks along the first dimension
            # This results in a tensor of shape [2, 61, 8, 8] (assuming all batches have 61 indices)
            resid_post = torch.stack(indexed_resid_posts)
            # print("Resid post", resid_post.shape)
            probe_out = einsum(
                "batch pos d_model, modes d_model rows cols options -> modes batch pos rows cols options",
                resid_post,
                linear_probe,
            )
            # print(probe_out.shape)

            # acc_blank = (probe_out[0].argmax(-1) == state_stack_one_hot[0].argmax(-1)).float().mean()
            # acc_color = ((probe_out[1].argmax(-1) == state_stack_one_hot[1].argmax(-1)) * state_stack_one_hot[1].sum(-1)).float().sum()/(state_stack_one_hot[1]).float().sum()

            

            probe_log_probs = probe_out.log_softmax(-1)
            probe_correct_log_probs = einops.reduce(
                probe_log_probs * state_stack_one_hot,
                "modes batch pos rows cols options -> modes pos rows cols",
                "mean"
            ) * one_hot_range # Multiply to correct for the mean over options
            # loss_even = -probe_correct_log_probs[0, 0::2].mean(0).sum() # note that "even" means odd in the game framing, since we offset by 5 moves lol
            # loss_odd = -probe_correct_log_probs[1, 1::2].mean(0).sum()
            loss_all = -probe_correct_log_probs[0, :].mean(0).sum()

            # if i % 1000 == 0:
            #     print(f"epoch {epoch}, batch {i}, acc_blank {acc_blank}, acc_color {acc_color}, loss_even {loss_even}, loss_odd {loss_odd}, loss_all {loss_all}")
            
            # loss = loss_even + loss_odd + loss_all
            loss = loss_all
            loss.backward() # it's important to do a single backward pass for mysterious PyTorch reasons, so we add up the losses - it's per mode and per square.

            if i % 100 == 0:
                print(f"epoch {epoch}, batch {i}, loss {loss}, lr {lr}")

            optimiser.step()
            optimiser.zero_grad()
            current_iter += batch_size

    checkpoint = {
        "linear_probe": linear_probe,
        "layer": layer,
        "config_name": config.linear_probe_name,
        "final_loss": loss,
        "model_name": model_name,
    }
    torch.save(checkpoint, linear_probe_name)

for i in range(0, n_layers, 2):
    train_linear_probe(i)
# train_linear_probe(12)

torch.Size([1, 512, 8, 8, 13])
torch.Size([32965, 61])


  0%|          | 1/10000 [00:01<4:03:01,  1.46s/it]

epoch 0, batch 0, loss 164.5199737548828, lr 0.0003


  1%|          | 102/10000 [00:16<23:36,  6.99it/s]

epoch 0, batch 100, loss 125.87535095214844, lr 0.00029729999999999996


...

epoch 0, batch 100, loss 109.36492919921875, lr 0.00029729999999999996


  2%|▏         | 202/10000 [11:29<27:40,  5.90it/s]

epoch 0, batch 200, loss 72.98895263671875, lr 0.00029459999999999995


  2%|▏         | 226/10000 [11:33<8:19:51,  3.07s/it]


KeyboardInterrupt: 